<a href="https://colab.research.google.com/github/seeedata/BADA/blob/main/2%ED%95%99%EA%B8%B0%20%EC%84%B8%EC%85%98%20%EA%B3%BC%EC%A0%9C/%22%EC%97%90%EC%BD%94%ED%94%84%EB%A1%9C%22%20%EB%84%A4%EC%9D%B4%EB%B2%84%EB%89%B4%EC%8A%A4%20%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 1

"에코프로"와 관련된 네이버 기사를 가져와 데이터프레임에 정리하시오.

칼럼: 제목, 기사 날짜, 내용

기간: 2023년 3월 1일 ~ 9월 27일

# import

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import pandas as pd

# 뉴스 기사 url 가져오기

In [ ]:
#뉴스 기사 링크를 리스트로 추출

def giveme_newslink():

    #기사 날짜 범위
    ds = '2023.03.01'
    de = '2023.09.27'

    #시작 페이지
    page = 1

    newslink_list = []

    while page <= 10000:

        url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%97%90%EC%BD%94%ED%94%84%EB%A1%9C&ds={ds}&de={de}&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{ds.replace('.','')}to{de.replace('.','')}&start={page}"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Whale/3.22.205.26 Safari/537.36'}

        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "lxml")
        a_tag = soup.find_all("a")

        for a in a_tag:
            class_list = a.get("class", [])  #a 태그 안에 있는 클래스 목록

            #조건1. a 태그 안의 href가 있어야 함
            #조건2. href 링크 안에 "article", "news.naver" 단어가 있어야 함 -> 뉴스 기사가 아닌 링크, 네이버뉴스 홈페이지로 연결되지 않는 기사가 크롤링되는 것을 방지
            #조건3. "sub_txt", "elss", "sub_tit" 클래스가 아니어야 함. -> 관련된 뉴스 크롤링 방지
            if ("href" in a.attrs) and ("article" in a["href"]) and ("news.naver" in a["href"]) and ("sub_txt" not in class_list) and ("elss" not in class_list) and ("sub_tit" not in class_list):
                newslink_list.append(a["href"])

        page += 10  #다음 페이지로 이동

        time.sleep(1.5)

    return newslink_list

In [ ]:
newslink_list = giveme_newslink()
newslink_list

['https://n.news.naver.com/mnews/article/003/0012113849?sid=101',
 'https://n.news.naver.com/mnews/article/417/0000951289?sid=101',
 'https://n.news.naver.com/mnews/article/366/0000935767?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005584825?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004896842?sid=101',
 'https://n.news.naver.com/mnews/article/366/0000935756?sid=101',
 'https://n.news.naver.com/mnews/article/366/0000935578?sid=101',
 'https://n.news.naver.com/mnews/article/119/0002752871?sid=101',
 'https://n.news.naver.com/mnews/article/293/0000047842?sid=101',
 'https://n.news.naver.com/mnews/article/421/0007073758?sid=101',
 'https://n.news.naver.com/mnews/article/215/0001126722?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004895927?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005583472?sid=101',
 'https://n.news.naver.com/mnews/article/009/0005191966?sid=101',
 'https://n.news.naver.com/mnews/article/243/0000050678?sid=101',
 'https://

In [ ]:
len(newslink_list)

5598

In [ ]:
#수집한 뉴스 링크가 날아가는 것을 방지하기 위해 데이터프레임으로 만든 후 csv 파일로 저장

newslink_df = pd.DataFrame()

for i in range(len(newslink_list)):
    row = pd.DataFrame({'Link': [newslink_list[i]]})  #각 링크를 'Link' 열에 넣기 위해 딕셔너리 형태로 생성
    newslink_df = pd.concat([newslink_df, row], ignore_index=True)  #데이터프레임을 병합

print(newslink_df)

                                                   Link
0     https://n.news.naver.com/mnews/article/003/001...
1     https://n.news.naver.com/mnews/article/417/000...
2     https://n.news.naver.com/mnews/article/366/000...
3     https://n.news.naver.com/mnews/article/018/000...
4     https://n.news.naver.com/mnews/article/015/000...
...                                                 ...
5593  https://n.news.naver.com/mnews/article/421/000...
5594  https://n.news.naver.com/mnews/article/029/000...
5595  https://n.news.naver.com/mnews/article/015/000...
5596  https://n.news.naver.com/mnews/article/468/000...
5597  https://n.news.naver.com/mnews/article/018/000...

[5598 rows x 1 columns]


In [ ]:
newslink_df.to_csv('C:/Users/이세은/Desktop/BADA/1005/뉴스링크_임시저장.csv')

# 각 뉴스 기사 크롤링

In [ ]:
def crawling(index):  #index: newslink_list의 인덱스

    art_dic = {}  #최종 추출 결과를 저장할 딕셔너리

    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    url = newslink_list[index]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Whale/3.22.205.26 Safari/537.36'}

    html = requests.get(url, headers = headers)
    soup = BeautifulSoup(html.text, "lxml")


    #제목 수집
    title = soup.select(title_selector)
    title_list = [t.text for t in title]
    #title_list에 있는 각 요소의 텍스트를 이어붙여서 하나의 문자열로 만든다.
    title_str = "".join(title_list)

    #날짜 수집
    date = soup.select(date_selector)
    date_list = [d.text[:11] for d in date]  #시간은 저장 안함
    date_str = "".join(date_list)

    #본문 수집
    main = soup.select(main_selector)
    main_list = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()  #m_text의 앞뒤 공백 제거
        main_list.append(m_text)
    main_str = "".join(main_list)


    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str

    return art_dic

In [ ]:
artdic_list = []

for i in tqdm(range(len(newslink_list))):
    art_dic = crawling(i)
    artdic_list.append(art_dic)

artdic_list

  0%|          | 0/5598 [00:00<?, ?it/s]

[{'title': "두산로보 이어 에코프로머티리얼즈 등판…'큰놈'들이 온다",
  'date': '2023.09.26.',
  'main': '두산로보틱스, 내달 5일 코스피 상장…증거금 \'33兆\' 모아서울보증보험·에코프로머티리얼즈 11월 코스피 입성 예정\n\n\n\n[서울=뉴시스] 김선웅 기자 = 26일 코스피가 전 거래일(2495.76)보다 2.21포인트(0.08%) 하락한 2493.64, 코스닥지수는 전 거래일(839.17)보다 2.63포인트(0.31%) 오른 841.80에 거래를 시작했다. 서울 외환시장에서 원·달러 환율은 전 거래일(1336.5원)보다 3.3원 높아진 1339.8원에 개장했다. 26일 오전 서울 중구 하나은행 딜링룸에서 딜러들이 업무를 수행하고 있다. 2023.09.26. mangusta@newsis.com[서울=뉴시스] 김경택 기자 = 두산로보틱스가 다음 달 5일 코스피 시장에 상장할 예정인 가운데 몸값이 최대 3조2700억원에 달하는 에코프로머티리얼즈가 코스피 입성에 출사표를 던졌다. 공모 청약에서 흥행을 기록한 두산로보틱스에 이어 서울보증보험, 에코프로머티리얼즈 등 조(兆)단위 새내기주들이 상장 초읽기에 돌입하면서 연말 기업공개(IPO) 시장이 모처럼 활기를 띨 것이란 기대가 커지고 있다.26일 금융투자업계에 따르면 국내 최대 전구체 생산 기업 에코프로머티리얼즈가 전날 금융위원회에 증권신고서를 제출하고 유가증권시장 상장을 위한 본격 공모 절차에 돌입했다.지난 2017년 출범한 에코프로머티리얼즈는 2004년 전구체 기술 연구개발과 함께 이차전지 핵심 소재 산업을 개척해 온 기업이다. 회사는 2006년 국내 첫 하이니켈 NC(A) 전구체, 2014년 세계 최초 NCM811 전구체와 2016년 NCM9½½ 전구체 개발에 성공했다. 현재는 전기차(EV) 등에 탑재되는 하이니켈 전구체를 주력으로 생산하고 있다.에코프로머티리얼즈의 예상 공모 금액은 5240억~6659억원 수준으로 상장 후 시가총액은 공모가 상단 기준 3조2700억원에

In [ ]:
art_df = pd.DataFrame(artdic_list)
art_df

,title,date,main
0,두산로보 이어 에코프로머티리얼즈 등판…'큰놈'들이 온다,2023.09.26.,"두산로보틱스, 내달 5일 코스피 상장…증거금 '33兆' 모아서울보증보험·에코프로머티..."
1,"""에코프로 사형제 될까"" 3조 에코프로머티리얼즈, 코스피 시동",2023.09.26.,에코프로머티리얼즈가 11월 코스피 상장을 목표로 IPO에 도전장을 내밀었다. 사진은...
2,상장 앞두고 이동채·가족회사 지분 580억 소각한 에코프로머티리얼즈,2023.09.27.,이동채 40만주·데이지파트너스 85.6만주 무상 증여이 회장 가족 주식 정체 처음 ...
3,"에코프로머티리얼즈, IPO로 그룹주 살릴까",2023.09.27.,에코프로 그룹주 9월 들어 20%대 하락테슬라 쇼크·양극재 판가 하락…'악재' 겹쳐...
4,"두산로보, 서울보증보험에 이어 에코프로머티리얼즈...미래에셋 대형 IPO석권",2023.09.27.,"미래에셋증권, ECM부분 1위 예상이 기사는 09월 26일 15:13 마켓인사이트에..."
...,...,...,...
5593,"""에코프로도 당했다""…미즈호·CDPQ 등 '불법 공매도' 무더기 적발",2023.07.14.,"주식 빌리지 않고 선매도 하는 '무차입 공매도' 혐의 회사측 ""고의 없었다, 착오였..."
5594,`황제주` 넘보는 에코프로…공매도와 후려치기에도 개미는 간다,2023.07.09.,에코프로 본사. 연합뉴스 에코프로가 연일 급등하며 코스닥 '황제주' 자리를 넘...
5595,"""이게 다 에코프로 때문""…증권사 떠나는 애널들 [돈앤톡]",2023.07.14.,"여전히 떠나는 애널들…왜""노력 대비 보상 적다"" 금전적 불만""공매도 결탁했냐"" 사회..."
5596,에코프로 ‘개미 vs 공매도’ 힘겨루기…최종 승자는?,2023.07.09.,사진|에코프로[스포츠서울 | 홍성효기자] 공매도 세력과 개인투자자 간 힘겨루기가 지...


In [ ]:
art_df.to_csv("C:/Users/이세은/Desktop/BADA/1005/에코프로_크롤링_결과1.csv", index=False)